## **Single Shot Detection**

Necessary Installations

For the object detection setup, necessary installation, tensoflow-gpu 1.15 and numpy 1.16 and other necessary libraries. Restart the Run time after installation

In [0]:
!pip install --upgrade tensorflow-gpu==1.15
!pip install numpy==1.16
!pip install pillow lxml jupyter matplotlib cython

     |████████████████████████████████| 411.5MB 30kB/s 
     |████████████████████████████████| 3.8MB 48.3MB/s 
     |████████████████████████████████| 512kB 46.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=99a8e934c731fca21ed0353a41b5a634876a788e07ce4b9016489c1f3a08f5d3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc4 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uni

Make sure to restart the runtime before going further


##   Downloading Tensoflow proto buffer

In [0]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

Selecting previously unselected package python-bs4.
(Reading database ... 144429 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

Guidelines:
This notebook has 3 parts
1. VISUALIZING THE RESULTS ON TENSORBOARD
2. RUNNING THE TRAINING
3. INFERENCING AND COUNTING NUMBER OF PRODUCTS

As the model is being trained and saved, you can run 1 and 3 sections. if you wish to start training from scratch create a new folder and upload the sample train data, test data and annotations.txt file in the following directory tree.

  

```

RandomSampleOf100Images.py  has code to sample train data.

```
#Directory guide
```

>gdrive
    >My Drive
      >object_detection
          >data
              >images #files in this folder needs to be uploaded
                  > train 
                  > test
                  > annotations.txt
          > models

```





# 1. VISUALIZING THE RESULTS ON TENSORBOARD

Run the following cells, a link is generated in the second cell output, clicking on that will open a tensorboard window.Past saved results can be viewed

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
#the logs that are created while training 
LOG_DIR = "training_1/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Need to mount the google drive to save data, and object detetion tensorflow models. User credentials are provided in the cell

In [0]:
from google.colab import drive
drive.mount('/gdrive')
# google credentials for this account 
                                    


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


# 2 .RUNNING THE TRAINING

In [0]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET
import tensorflow.compat.v1 as tf
import cv2 
from PIL import Image
from collections import namedtuple, OrderedDict
from google.colab import files

In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
%cd /gdrive/'My Drive'/

/gdrive/My Drive


Uncomment the following cell and create directory only if you want to work from scratch

In [0]:
#%mkdir object_detection

mkdir: cannot create directory ‘object_detection’: File exists


In [0]:
%cd /gdrive/'My Drive'/object_detection

/gdrive/My Drive/object_detection


At /gdrive/My Drive/object_detection location 

1.   Create a folder data(cmd : !mkdir data)
2.   inside data create another floder images, and upload train, test and annotations file to this folder



All the necessary models from the tensorflow has been downloaded, if you wish to start afresh uncomment and run the cell. before doing that delete the models folder from the drive

In [0]:
#!git clone --q https://github.com/tensorflow/models.git

In [0]:
%cd models/research/

/gdrive/My Drive/object_detection/models/research


Here at models/research/ we compile the proto buffers and set enironmental variables with research and slim path

In [0]:
import os
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':./:./slim/'

Run the next cell to see if the protos buffer and ENV variables are properly set

In [0]:
!python3 object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

# **Data Preparation**
The annotations provided were not adhering to the SSD annotations input in tensorflow

Annotations from github were:

eg. (image name, number of products, x (left top), y(left top), width(bounding box), height(bounding box), class)
			
The above format is converted to:

eg. (image name, width, height, class, xmin, ymin, xmax, ymax)	

xmin, ymin are same as x, y from github annotations.

xmax is calculated as.

```
xmax = xmin + 0.95*(widht of bounding box)
```
Have taken xmin+width of boouding box, as the boxes are adjacent to each other, there was a slight overlap in the bouding box region. To avoid that i took 0.95 times the width.
```
ymax = ymin + height of bounding box
```
**Note:
width -- full image width(not bounding box)
height -- full image height(not bounding box)

**Number of Classes:**

In the given data we have 11 products of similar shape and sizes.
		Our task is to predict whether there is a product or not in an image.


Since, we want to count the number of items, all we need to do is to classify the bounding box as empty or not. If it's not empty we would count it. So, I have put all 11 products under one class and 'empty space', as another  class(by default).

The annotations text file is converted to a train_labels.csv and test_labels.csv

In [0]:
#file created here should save file in annotation as train_label.csv and train_label.csv
import pandas as pd
from PIL import Image
import os
import pandas as pd

def text_annot_to_csv(IMAGE_PATH):
    img_annot = []
    train_annot = []
    test_annot = []
    train_set = set(os.listdir(IMAGE_PATH + "/train/"))
    test_set = set(os.listdir(IMAGE_PATH + "/test/"))   
    label_set = set()
    with open(IMAGE_PATH + "/annotations.txt", "r") as a_file:
        for line in a_file:
            img_annot = line.rstrip('\n').split(' ')
            filename ,num = img_annot[:2]
            if (filename in train_set) or (filename in test_set):
              annot = img_annot[2:]
              for i in range(0, len(annot), 5):
                  xmin = int(annot[i:i+5][0]) 
                  ymin = int(annot[i:i+5][1])
                  xmax = xmin + int((0.95)*int(annot[i:i+5][2]))
                  ymax = ymin + int(annot[i:i+5][3])
                  label = 'box'
                  label_set.add(label)
                  if filename in train_set:
                      im = Image.open(IMAGE_PATH + '/train/' + filename )
                      width, height = im.size
                      train_annot.append((filename, width, height, label, xmin, ymin, xmax, ymax))
                  else:
                      im = Image.open(IMAGE_PATH + '/test/' + filename )
                      width, height = im.size
                      test_annot.append((filename, width, height, label, xmin, ymin, xmax, ymax))

    label_map_path = IMAGE_PATH + "label_map.pbtxt"
    pbtxt_content = ""
    for i, label in enumerate(label_set):
        pbtxt_content = ( pbtxt_content + "item {{\n   name: '{0}' \n    id: {1}\n}}\n\n".format(label, i+1))
    pbtxt_content = pbtxt_content.strip()
    with open(label_map_path, "w") as f:
        f.write(pbtxt_content)
    return train_annot, test_annot
	
def main():

	IMAGE_PATH = '/gdrive/My Drive/object_detection/data/images/'
						
	OUTPUT_DIR = '/gdrive/My Drive/object_detection/data/'

	train, test = text_annot_to_csv(IMAGE_PATH)
	
	column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
	train_df = pd.DataFrame(train, columns=column_name)
	test_df = pd.DataFrame(test, columns= column_name)
	train_df.to_csv(OUTPUT_DIR + '/train_labels.csv', index=None)
	test_df.to_csv(OUTPUT_DIR + '/test_labels.csv', index=None)
	print("---Successfully converted---")
	
if __name__ == '__main__':
	main()

---Successfully converted---


**Generating TF Recodrs.**

Tensorflow accepts the data as TFRecords data.record. TFRecord is a binary file that runs fast with low memory usage.

In [0]:

from object_detection.utils import dataset_util
import pandas as pd
import io
import pandas as pd
import tensorflow as tf
import sys
from absl import app
from absl import flags

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict
data_base_url = '/gdrive/My Drive/object_detection/data/'

image_dir = data_base_url +'images/'

def class_text_to_int(row_label):
	if row_label == 'box':  
		return 1
	else:
		return None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
	  encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size
	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(row['xmin'] / width)
		xmaxs.append(row['xmax'] / width)
		ymins.append(row['ymin'] / height)
		ymaxs.append(row['ymax'] / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(class_text_to_int(row['class']))

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
	return tf_example


for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(data_base_url + csv + '.record')
  if csv == 'train_labels':
    path = os.path.join(image_dir+ '/train' )
  else:
    path = os.path.join(image_dir+ '/test' )
  examples = pd.read_csv(data_base_url + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))

Successfully created the TFRecords: /gdrive/My Drive/object_detection/data/train_labels.record
Successfully created the TFRecords: /gdrive/My Drive/object_detection/data/test_labels.record


**Selecting and downloading the pre-trained models**

In [0]:
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'ssd_inception_v2_coco':{
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
    },
    
}

selected_model = 'ssd_inception_v2_coco'

In [0]:
#the distination folder where the model will be saved
#change this if you have a different working dir
DEST_DIR = '/gdrive/My Drive/object_detection/models/research/pretrained_model'

MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

**Configuring the training Pipeline**

Running the below cell gives a default config file of the pretrained model chosen. 

In [0]:
!cat object_detection/samples/configs/ssd_inception_v2_coco.config


# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 2.0
        reduce_boxes_in_lowest_layer: tru

Copy the output from the above cell and paste it below the command. Running this will let you modify the config file.

Change the num of classes in model from 90 to 1.

At anchor genrator, remove all aspect ratios and give only one value i.e 2.

Paths to be modified are.

1. fine tune checkpoint
2. train input reader
3. label map path
4. eval input reader
5. label map path

At train config, add data augmentation to imporve the results

After changing the above fields run the cell



In [0]:
%%writefile object_detection/samples/configs/ssd_inception_v2_coco.config

# SSD with Inception v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 2.0
        reduce_boxes_in_lowest_layer: true
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: true
        dropout_keep_probability: 0.8
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_inception_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 24
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "/gdrive/My Drive/object_detection/models/research/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/object_detection/data/train_labels.record"
  }
  label_map_path: "/gdrive/My Drive/object_detection/data/images/label_map.pbtxt"
}

eval_config: {
  num_examples: 400
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  num_visualizations: 20
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/object_detection/data/test_labels.record"
  }
  label_map_path: "/gdrive/My Drive/object_detection/data/images/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting object_detection/samples/configs/ssd_inception_v2_coco.config


Running the next two cells will allow use to visualize the results in tensorboard

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-05-06 05:10:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.86.21.186, 3.213.66.77, 52.4.202.19, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.86.21.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.5’

ngrok-stable-linux- 100%[===================>]  13.13M  18.5MB/s    in 0.7s    

2020-05-06 05:10:41 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.5’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


The generated link after running the below cell will open a tensorboard window in a new tab.

In [0]:
#the logs that are created while training 
LOG_DIR = "training_1/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://f4a068af.ngrok.io


**Training**

Running the cell will start the training, this will take a while

In [0]:
!python3 object_detection/model_main.py \
    --pipeline_config_path=/gdrive/My\ Drive/object_detection/models/research/object_detection/samples/configs/ssd_inception_v2_coco.config \
    --model_dir=training_1/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0506 05:11:11.970101 140173809837952 module_wrapper.py:139] From /gdrive/My Drive/object_detection/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0506 05:11:11.975579 140173809837952 module_wrapper.py:139] From /gdrive/My Drive/object_detection/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0506 05:11:11.975744 140173809837952 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0506 05:11:11.975898 140173809837952

**Export the trained model.**

Running export_inference_graph.py will convert the model to a frozen model frozen_inference_graph.pb that we can use for inference. 

In [0]:

output_directory = './fine_tuned_model_1'

lst = os.listdir('training_1')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('training_1', last_model)

In [0]:
!python /gdrive/'My Drive'/object_detection/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/gdrive/My\ Drive/object_detection/models/research/object_detection/samples/configs/ssd_inception_v2_coco.config \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0506 10:57:56.895951 139664240314240 module_wrapper.py:139] From /gdrive/My Drive/object_detection/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0506 10:57:56.905526 139664240314240 module_wrapper.py:139] From /gdrive/My Drive/object_detection/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0506 10:57:56.909390 139664240314240 module_wrapper.py:139] From /gdrive/My Drive/object_detection/models/research/object_detection/exporter.py:121: The na

# 3. MODEL INFERENCING AND COUNTING THE PRODUCTS IN THE IMAGE

**Inferencing the performance of the model and counting the number of products present**


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from pathlib import Path
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1
# Patch the location of gfile
tf.gfile = tf.io.gfile

Running the below cells will load frozen inference graph from the fine tuned model which was previously saved after training.

In [0]:

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/gdrive/My Drive/object_detection/models/research/fine_tuned_model_1/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = "/gdrive/My Drive/object_detection/data/images/label_map.pbtxt"

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = Path('/gdrive/My Drive/object_detection/data/images/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.JPG")))
TEST_IMAGE_PATHS[:1]

[PosixPath('/gdrive/My Drive/object_detection/data/images/test/C1_P02_N1_S5_1.JPG')]

In [0]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [0]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
                        
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

**Counting the products in the test images**

The below cell will generate output for each image, this output contains detection class, detection scores, detection boxes.

In order to know the number of prodcuts in each test image, we set the threshold on detection scores to 0.5, The total number of scores whose value greater than 0.5 is considered as a total number of prodcuts in the given image.

The first cell wil take a while to run, the subsequent cells will generate a json file having shelf_image_filename and product count as key and value.

In [0]:
import numpy as np
shelf_images = {}
shelf_img_scores = {}
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    name = str(image_path).split('/')[-1]
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    shelf_images['shelf_image_' + name] = int(np.sum(output_dict['detection_scores'] > 0.5))
    shelf_img_scores['shelf_image_' + name] = list(map(float, output_dict['detection_scores']))

In [0]:
import json
with open('/gdrive/My Drive/image2products.json', 'w') as fp:
    json.dump(shelf_images, fp)
#with open('/gdrive/My Drive/scores2images.json', 'w') as ff:
#    json.dump(shelf_img_scores, ff)

In [0]:
%cd '/gdrive/My Drive/'

/gdrive/My Drive


In [0]:
from google.colab import files
files.download('image2products.json')


In [0]:
#files.download('scores2images.json') 

**Run this cell to see the sample test images, where the boudning boxes are drawn.**

In [0]:
%matplotlib inline
import numpy as np
IMAGE_SIZE = (10, 8)
shelf_images = {}
for image_path in TEST_IMAGE_PATHS[10:21]:
    image = Image.open(image_path)
    name = str(image_path).split('/')[-1]
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)